##### SQL 삽입 (p.15)

1) DB-API 사용 예제

In [ ]:
from django.shortcuts import render

def update_board(request):
    ...
	with dbconn.cursor() as curs:
		name = request.POST.get('name', '')
		content_id = request.POST.get('content_id', '')
		
		# 외부 입력값으로 부터 안전한 매개변수 화된 쿼리를 생성 한다.
		sql_query = 'update board set name=%s where content_id=%s'
		
		# 사용자의 입력 값을 매개변수 화된 쿼리에 바인딩 하여 실행되므로
		# 안전하다.
		curs.execute(sql_query, (name, content_id))
		curs.commit()
		return render(request, '/success.html')

2) ORM 사용 예제

In [ ]:
from django.shortcuts import render
from app.models import Member

def member_search(request):
    # 외부로부터 입력 값을 가져온다.
    name = request.POST.get('name', '')

    # 외부 입력 값을 raw()함수 실행 시 바인딩 변수로 사용하여 쿼리 구조가
    # 변경되지 않도록 한다.(list 형은 %s, dictionary 형은 %(key)s를 사용)
    query='select * from member where name=%s'
    
    # 인자화된 쿼리문을 사용하여 raw()함수를 사용하여 안전하다.
    data = Member.objects.raw(query, [name])
    return render(request, '/member_list.html', {'member_list':data})

##### 코드삽입 (p.20)

1) eval()함수 사용 예제

In [ ]:
from django.shortcuts import render

def route(request):
    message = request.POST.get('message', '')
    
    # 사용자 입력을 영문, 숫자로 제한하는 예로. 특수문자가 포함되어
    # 있을 경우 에러 메시지를 리턴 한다.
    
    if message.isalnum():
        ret = eval(message)
        return render(request, '/success.html', {'data':ret})
    ......

2) exec()함수 사용 예제

In [ ]:
from django.shortcuts import render

WHITE_LIST = ['get_friends_list', 'get_address', 'get_phone_number']

def request_rest_api(request):
    function_name = request.POST.get('function_name', '')

    # 함수 명을 화이트리스트로 제한
    if function_name in WHITE_LIST:
        ret = exec('{}()'.format(function_name))
        return render(request, '/success', {'data':ret})
        
    return render(request, '/error', {'error':'허용되지 않은 함수입니다.'})

##### 경로 조작 및 자원 삽입 (p.24)

1) 경로 조작 예제

In [ ]:
import os
from django.shortcuts import render

def get_info(request):
    # 외부 입력 값으로 받은 파일 이름은 검증하여 사용한다.
    request_file = request.POST.get('request_file')

    filename, file_ext = os.path.splitext(request_file)
    file_ext = file_ext.lower()
   
    if file_ext not in ['.txt', '.csv']:
        return render(request, '/error.html', {'error':'파일을 열수 없습니다.'})
    
    # 파일 명에서 경로 조작 문자열을 필터링 한다.
    filename = filename.replace('.', '')
    filename = filename.replace('/', '')
    filename = filename.replace('\\', '')

    with open(filename + file_ext) as f:
        data = f.read()

    return render(request, '/success.html', {'data':data})

2) 자원 삽입 예제

In [ ]:
import socket
from django.shortcuts import render

ALLOW_PORT = [4000, 6000, 9000]

def get_info(request):
    port = int(request.POST.get('port'))

    # 사용 가능한 포트 번호를 화이트리스트로 제한
    if port not in ALLOW_PORT:
        return render(request, '/error', {'error':'소켓연결 실패'})
   
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('127.0.0.1', port))
        ......
    return render(request, '/success')

##### 크로스사이트 스크립트(XSS) (p.29)

1) Django 예제

In [ ]:
from django.shortcuts import render
from django.utils.safestring import mark_safe

def profile_link(request):
    # 외부 입력값을 검증 없이 HTML 태그 생성의 인자로 사용
    profile_url = request.POST.get('profile_url')
    profile_name = requst.POST.get('profile_name')
    
    object_link = '<a href="{}">{}</a>'.format(profile_url, profile_name)
    # 신뢰 할 수 없는 데이터에 대해서는 mark_safe 함수를 사용하지 않는다

    return render(request, 'my_profile.html',{'object_link':object_link})

In [ ]:
<!doctype html>
<html>
    <body>
        <div class="content">
            {% autoescape on %}
                // autoescape on로 해당 블록내의 데이터는 XSS 공격에
                {{ content }}
            {% endautoescape %}
        </div>
        <div class="content2">
            //검증되지 않은 데이터에는 safe 필터를 사용하지 않는다.
            {{ content}}
        </div>
    </body>
</html>

2) Flak 예제

In [ ]:
import html
from flask import Flask, request, render_template

@app.route('/search', methods=['POST'])
def search():
    search_keyword = request.form.get('search_keyword')
    
    # 동적 웹 페이지 생성에 사용되는 데이터는
    # HTML 엔티티코드로 치환하여 표현해야한다
    escape_keyword = html.escape(search_keyword)
    return render_template('search.html', search_keyword=escape_keyword)

##### 운영체제 명령어 삽입 (p.34)

In [ ]:
import os
from django.shortcuts import render

ALLOW_PROGRAM = ['notepad', 'calc']

def execute_command(request):
    app_name_string = request.POST.get('app_name','')

    # 입력받은 파라미터를 사용가능한 시스템 명령어 일부로 제한하여 사용
    if app_name_string not in ALLOW_PROGRAM:
        return render(request, '/error.html', {'ERROR':'허용되지 않은 프로그램입니다.'})
   
    os.system(app_name_string)
    return render(request, '/success.html')

In [ ]:
import subprocess
from django.shortcuts import render

def execute_command(request):
    date = request.POST.get('date','')

    # 명령어를 추가로 실행 또는 다른 명령이 실행될 수 있는 키워드에
    # 대한 예외처리
    for word in ['|', ';', '&', ':', '>', '<', '`', '\\', '!']:
        date = date.replace(word, "")

    # shell=True 옵션은 제거 하고 명령과 인자를 배열로 입력
    subprocess.run(["cmd", "/c", "backuplog.bat", date])
    return render(request, '/success.html')

##### 위험한 형식 파일 업로드 (p.38)

In [ ]:
import os
from django.shortcuts import render
from django.core.files.storage import FileSystemStorage

# 업로드 하는 파일에 대한 개수, 크기, 확장자 제한
FILE_COUNT_LIMIT = 5
# 업로드 하는 파일의 최대 사이즈 제한 예 ) 5MB - 5*1024*1024
FILE_SIZE_LIMIT = 5242880
# 허용하는 확장자는 화이트리스트로 관리한다.
WHITE_LIST_EXT = [
    '.jpg',
    '.jpeg'
]

def file_upload(request):
    # 파일 개수 제한
    if len(request.FILES) == 0 or len(request.FILES) > FILE_COUNT_LIMIT:
        return render(request, '/error.html', {'error': '파일 개수 초과'})
    
    for filename, upload_file in request.FILES.items():
        # 파일 타입 체크
        if upload_file.content_type != 'image/jpeg':
            return render(request, '/error.html', {'error': '파일 타입 오류'})
        # 파일 크기 제한
        if upload_file.size > FILE_SIZE_LIMIT:
            return render(request, '/error.html', {'error': '파일사이즈 오류'})
        # 파일 확장자 검사
        file_name, file_ext = os.path.splitext(upload_file.name)
        if file_ext.lower() not in WHITE_LIST_EXT:
            return render(request, '/error.html', {'error': '파일 타입 오류'})
        
    fs = FileSystemStorage(location='media/screenshot', base_url = 'media/screenshot')
    for upload_file in request.FILES.values():
        fs.save(upload_file.name, upload_file)
        
    return render(request, '/success.html', {'filename': filename})

##### 신뢰되지 않은 URL주소로 자동접속 연결 (p.41)

In [ ]:
from django.shortcuts import redirect

ALLOW_URL_LIST = [
    '127.0.0.1',
    '192.168.0.1',
    '192.168.0.100',
    'https://login.myservice.com',
    '/notice'
]

def redirect_url(request):
    url_string = request.POST.get('url', '')

    # 이동할 수 있는 URL 범위를 제한하여
    # 위험한 사이트의 접근을 차단하고 있다
    if url_string not in ALLOW_URL_LIST:
        return render(request, '/error.html', {'ERROR':'허용되지 않는 주소입니다.'})
    
    return redirect(url_string)

##### 부적절한 XML 외부 개체 참조 (p.44)

In [ ]:
from xml.sax import make_parser
from xml.sax.handler import feature_external_ges
from xml.dom.pulldom import parseString, START_ELEMENT
from django.shortcuts import render
from .model import comments

def get_xml(request):
    if request.method == "GET":
        data = comments.objects.all()
        com = data[0].comment
        return render(request, '/xml_view.html', {'com':com})
    
    elif request.method == "POST":
        parser = make_parser()
        parser.setFeature(feature_external_ges, False)
        doc = parseString(request.body.decode(‘utf-8’), parser=parser)
        for event, node in doc:
            if event == START_ELEMENT and node.tagName == “foo”:
                doc.expandNode(node)
                text = node.toxml()
        comments.objects.filter(id=1).update(comment=text);
        return render(request, '/xml_view.html')

##### XML 삽입 (p.46)

In [ ]:
from lxml import etree

def parse_xml(request):
    user_name = request.POST.get('user_name', '')

    parser = etree.XMLParser(resolve_entities=False)
    tree = etree.parse('user.xml', parser)
    root = tree.getroot()
    
    query = '/collection/users/user[@name = $paramname]/home/text()'
    #외부 입력값을 paramname으로 매개변수화 해서 사용
    elmts = root.xpath(query, paramname=user_name)
    return render(request, 'parse_xml.html', {'xml_element':elmts})

##### LDAP 삽입 (p.49)

In [ ]:
from ldap3 import Connection, Server, ALL
from ldap3.utils.conv import escape_filter_chars
from django.shortcuts import render

def ldap_query(request):
    search_keyword = request.POST.get('search_keyword','')

    dn = server.config['bind_dn']
    password = server.config['password']

    address = 'ldap.goodsource.com'
    server = Server(address, get_info=ALL)
    conn = Connection(server, dn, password, auto_bind=True )
    
    # 사용자의 입력에 필터링을 적용하여 공격에 사용될 수 있는 문자를
    # 이스케이프하고 있다
    escpae_keyword = escape_filter_chars(search_keyword)
    search_str = '(&(objectclass=%s))' % escpae_keyword

    conn.search('dc=company,dc=com', search_str, attributes=['sn', 'cn', 'address', 'mail', 'mobile', 'uid'])
    return render(request, '/ldap_query_response.html', {'ldap':conn.entries})

#####  크로스사이트 요청 위조(CSRF) (p.52)

1. Django 프레임워크 사용 - 미들웨어 설정(settings.py) 사례

In [ ]:
MIDDLEWARE = [
    'django.contrib.sessions.middleware.SessionMiddleware',
    # MIDDLEWARE 목록에서 csrf 항목을 활성화 한다.
    'django.middleware.csrf.CsrfViewMiddleware',
    'django.contrib.auth.middleware.AuthenticationMiddleware',
    'django.contrib.messages.middleware.MessageMiddleware',
    'django.middleware.locale.LocaleMiddleware',
    ......
]

2. Django 프레임워크 사용 - 뷰 기능 설정(views.py) 사례

In [ ]:
from django.shortcuts import render
from django.views.decorators.csrf import csrf_exempt

# csrf_exempt 데코레이터를 삭제하거나 주석처리한다.
# @csrf_exempt
def pay_to_point(request):
    user_id = request.POST.get('user_id', '')
    pay = request.POST.get('pay', '')
    product_info = request.POST.get('product_info', '')
    
    ret = pay(user_id, pay, product_info)
    
    return render(request, '/view_wallet.html', {'wallet':ret})

3. Django 프레임워크 사용 - 템플릿 설정 사례

In [ ]:
<!--html page-->
<form action="" method="POST">
    {% csrf_token %} <!--csrf_token 사용->
    <table>
        {{form.as_table}}
    </table>
    <input type="submit"/>
</form>

4. Flask 프레임워크 사용 - app 설정 사례

In [ ]:
from flask import Flask
from flask_wtf.csrf import CSRFProtect

# CSRF 설정 추가
csrf = CSRFProtect(app)
app = Flask(__name__)
app.config[‘SECRET_KEY’] = os.environ.get('SECRET_KEY')
csrf.init_app(app)

5. Flask 프레임워크 사용 - 템플릿 설정 사례

In [ ]:
<form action="" method="POST">
    <!-- form 태그 내부에 csrf_token 적용-->
    <input type="hidden" name="csrf_token" value="{{ csrf_token }}" />
    <table>
        {{table}}
    </table>
    <input type="submit"/>
</form>

##### 서버사이드 요청 위조 (p.58)

In [ ]:
from django.shortcuts import render
import requests

# 도메인을 화이트리스트에 정의 할 경우 DNS rebinding 공격 등에
# 노출될 위험이 있어 신뢰 할 수 있는 자원에 대한 IP를 사용하여
# 검증하는 것이 좀더 안전하다.
ALLOW_SERVER_LIST = [
    'https://127.0.0.1/latest/',
    'https://192.168.0.1/user_data',
    'https://192.168.0.100/v1/public'
]

def call_third_party_api(request):
    addr = request.POST.get('address', '')

    # 사용자가 입력한 URL을 화이트리스트로 검증한 후 그 결과를 반환하여
    # 검증되지 않은 주소로 요청을 보내지 않도록 제한한다.
    if addr not in ALLOW_SERVER_LIST:
        return render(request, '/error.html', {error = '허용되지 않은 서버입니다.'})
    
    result = requests.get(addr).text
    return render(request, '/result.html', {'result':result})

##### HTTP 응답분할 (p.61)

In [ ]:
from django.http import HttpResponse

def route(request):
    content_type = request.POST.get('contnet-type')
    
    # 응답헤더에 포함될 수 있는 외부 입력값에 대해 개행문자를 제거한다.
    content_type = content_type.replace('\r', '')
    content_type = content_type.replace('\n', '')
    ......
    res = HttpResponse()
    res['Content-Type'] = content_type
    return res

##### 보안기능 결정에 사용되는 부적절한 입력값 (p.63)

In [ ]:
from django.shortcuts import render

def init_password(request):
    # 세션에서 권한 정보를 가져옴
    roll = request.session['roll']
    request_id = request.POST.get('user_id', '')
    request_mail = request.POST.get('user_email','')
    # 세션에서 가져온 권한이 관리자인지 비교
    if roll == 'admin':
        # 사용자의 패스워드 초기화 및 메일 발송 처리
        password_init_and_sendmail(request_id, request_mail)
        return render(request, '/sucess.html')
    else:
        return render(request, '/failed.html')

##### 포맷 스트링 삽입 (p.66)

In [ ]:
from django.shortcuts import render
AUTHENTICATE_KEY = 'Passw0rd'

def make_user_message(request):
    user_info = get_user_info(request.POST.get('user_id', ''))
    
    # 사용자가 입력한 문자열을 포맷 문자열로 사용하지 않아 안전하다
    message = 'user name is {}'.format(user_info.name)
    
    return render(request, '/user_page', {'message':message})